In [1]:
!git clone https://github.com/ax-le/data_for_clap_demo # to get necessary data

Clonage dans 'data_for_clap_demo'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 3), reused 17 (delta 2), pack-reused 0
Réception d'objets: 100% (18/18), 6.03 Mio | 6.48 Mio/s, fait.
Résolution des deltas: 100% (3/3), fait.


In [2]:
# Some important imports
from transformers import ClapModel, ClapProcessor # CLAP model
import numpy as np # Data manipulation tool
import torch # Neural network development module
import torchaudio.transforms as T # Transforms for audio (spectrogram and everything)
from torchaudio import load # Loader for audio files
from librosa import power_to_db
from sklearn.metrics import pairwise_distances  # Cosine similarity

In [3]:
import pathlib
pathlib.Path("./cache_model").mkdir(exist_ok=True) # Create a cache folder here for the notebook to run

## change this to your own cache directory to download the models
cache_dir = './cache_model'

device=torch.device("cuda" if torch.cuda.is_available() else "cpu") # Specify the model to make inference
## You can also set this parameter manually (if you have only one GPU, use "cuda:0", or use "cpu" if you don't have a GPU)

In [4]:
# Load the models
cpt1 = "laion/clap-htsat-unfused" # Name of the model
processor_general = ClapProcessor.from_pretrained(cpt1)
model_general = ClapModel.from_pretrained(cpt1,cache_dir=cache_dir).to(device)

# Prints the model
# model_general.eval()

config.json:   0%|          | 0.00/5.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

KeyboardInterrupt: 

Load and prepare an audio file

In [ ]:
## change this to your own audio file
audiofile = './data_for_clap_demo/example.flac'

# Load the audio file
waveform, base_sr = load(audiofile)

# check the sampling rate and resample to 48000 if needed (the models were trained on 48kHz audio)
clap_sr = 48000
if base_sr != clap_sr:
    resampler = T.Resample(base_sr, clap_sr)
    waveform = resampler(waveform)

# check the number of channels and convert to mono if needed
if waveform.shape[0] > 1:
    waveform = torch.mean(waveform, dim=0, keepdim=False)
else:
    waveform = waveform.squeeze()

# Here we will only process the first 10 seconds of the audio file
waveform = waveform[:(10*clap_sr)]

# play the audio file
from IPython.display import Audio
Audio(waveform, rate=clap_sr)

In [ ]:
import matplotlib.pyplot as plt # Plotting module
# Plots the waveform
time_axis = torch.arange(0, waveform.shape[0]) / clap_sr
plt.plot(time_axis, waveform)
plt.title("Waveform of the signal")
plt.xlabel("Time (s)")
plt.show()

# Computes and plots the spectrogram
spectrogram = T.Spectrogram(n_fft=512, hop_length = 256)
spec = spectrogram(waveform)
plt.imshow(power_to_db(spec), origin="lower", aspect="auto", interpolation="nearest")
plt.title("Spectrogram (STFT) of the signal")
plt.ylabel("Frequency bin")
plt.show()

In [ ]:
## Generate the embeddings

#preprocess the audio
inputs_audio = processor_general(audios=waveform, sampling_rate=clap_sr,return_tensors="pt", padding=True)

# move the inputs to the GPU (inputs audio is a dictionary)        
for key, value in inputs_audio.items():
    inputs_audio[key] = value.to(device)

# run the inputs into the model 
with torch.inference_mode():
    audioembeddings = model_general.get_audio_features(**inputs_audio)

We will try to perform some zero-shot classification on this file. We heard some crickets, a car, maybe a bird. Let's try to classify these three categories in zero-shot by generating text embeddings corresponding to these classes. 

In [ ]:
## Here is the list of target "classes" we will use to compute the similarity scores.
## The model will now classify our audio example based on these classes.
# Note that we have added some very unrelated ones to see how the model behaves.

texts = ["Crickets are singing", "A car is passing", "A train is passing",
         "Birds are singing","A dog is barking", 
         "Someone is playing the piano", 
         "Someone is talking"]

Generate the text embeddings

In [ ]:
# Process the input texts ("classes")
inputs_text = processor_general(text=texts, return_tensors="pt", padding=True)
for key, value in inputs_text.items():
    inputs_text[key] = value.to(device)

# Classify
with torch.inference_mode():
    outputs_text = model_general.get_text_features(**inputs_text)

Zero-shot classification is performed by searching for the nearest neighbour between the audio embedding and text embeddings

In [ ]:
#print("Audio embeddings shape:",audioembeddings.shape) # Embedding of the audio
#print("Text embeddings shape:",outputs_text.shape) # Embedding of the text

# Compute the cosine similarity between the audio embedding and the text embeddings
cosine_similarity = 1-pairwise_distances(audioembeddings.cpu(), outputs_text.cpu(), metric="cosine")

print(f"Most similar text: {texts[np.argmax(cosine_similarity[0,:])]}\n")

# Print the similarity scores
print("In details, with similarity scores:")
for i, text in enumerate(texts):
    print(f"\t{text}: {cosine_similarity[0][i]:.3f}")

We developed a high level function to classify an audio example, for easeness of use. It is used in the cell below:

In [ ]:
import data_for_clap_demo.utils as utils
classification = utils.classify_this_audio_file(processor=processor_general, model=model_general, audio=waveform, text_embeddings=utils.esc_50_labels)
print(classification)

CLAP needs text labels to perform classification, so we used the ones from ESC-50, a standard dataset. Feel free to use your owns!

What about the ambiguous example presented in the slides?

In [ ]:
# Load the audio file
ambiguous_example, base_sr = load("./data_for_clap_demo/foley.wav")
ambiguous_example = torch.mean(ambiguous_example, dim=0, keepdim=False)
ambiguous_classification = utils.classify_this_audio_file(processor=processor_general, model=model_general, audio=ambiguous_example, 
                                                          text_embeddings=utils.esc_50_labels)
print(f"From the text labels of ESC-50, the ambiguous sound is classified as: {ambiguous_classification}")

From the labels of the ESC-50 dataset, CLAP classified the ambiguous example from the slides as a 'crackling fire', but the correct answer (plastic bag) is not present in the ESC-50 labels. What about classification between the proposed examples in the slides? (plastic bag,people stomping,crackling fire, or rain)

In [ ]:
ambiguous_classification = utils.classify_this_audio_file(processor=processor_general, model=model_general, audio=ambiguous_example, 
                                                          text_embeddings=["plastic bag", "people stomping","crackling fire", "rain"])
print(f'Classification between "plastic bag", "people stomping", "crackling fire", and "rain": {ambiguous_classification}')

Still failing (but it is a difficult example).

Finally, CLAP can also be used for the classification of multiple audio examples (such as a long dataset). As an example, we calssify 10 examples from the ESC-50 dataset (link to download: https://github.com/karolpiczak/ESC-50/tree/master)

In [ ]:
import os 

# Preprocess the dataset
path_to_dataset = "./data_for_clap_demo/audio_esc" # The path to the desired dataset (parent folder of audio files)
list_of_audiofiles = os.listdir(path_to_dataset)
duration = 10*clap_sr # 10 seconds
allwaves = []

for curfile in list_of_audiofiles:
    #open the files with torchaudio
    waveform, sr = load(f"{path_to_dataset}/{curfile}")    
    if sr != clap_sr:
        waveform = T.Resample(sr,clap_sr)(waveform)[0,:]

    # append the waveform to the list
    allwaves.append(waveform[:duration])

# concatenate all the waveforms
allwaves = np.stack(allwaves)

print(allwaves.shape)

inputs_audio = processor_general(audios=allwaves, sampling_rate=clap_sr,return_tensors="pt", padding=True)
        
for key, value in inputs_audio.items():
    inputs_audio[key] = value.to(device)

with torch.inference_mode():
    audio_embeddings_dataset = model_general.get_audio_features(**inputs_audio)

Now, classify all examples:

In [ ]:
text_embeddings=utils.esc_50_labels
inputs_text = processor_general(text=text_embeddings, return_tensors="pt", padding=True)
for key, value in inputs_text.items():
    inputs_text[key] = value.to(device)

# Classify
with torch.inference_mode():
    text_embeddings_clap = model_general.get_text_features(**inputs_text)

cosine_similarity = 1-pairwise_distances(audio_embeddings_dataset.cpu(), text_embeddings_clap.cpu(), metric="cosine")
for i in range(len(list_of_audiofiles)):
    print(f"Embedding for file {i}: {text_embeddings[np.argmax(cosine_similarity[i,:])]}")